This notebook explores possibilities to perform bulk actions on the SSH Open Marketplace. Start with mass rejection of ingested items (CLARIN resource families).

In [ ]:
import pandas as pd #to manage dataframes
import json #to manage json objects
import requests #to interact with the API
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import eval as eva, helper as hel

In [ ]:
mpdata = mpd()
utils=hel.Util()
check=eva.URLCheck()

In [ ]:
import urllib
from urllib.parse import urlparse
bearer=mpdata.checkCredentials()
headers = {'Authorization':bearer}
status = 'ingested'
source = 'CLARIN Resource Families'

# The 'urllib.parse.quote(source)' added for the value of 'f.source' is not mandatory, 
# but it is a good pratice to encode strings used to build URLs
# when it may contain blanks, as it is in this case, or other special characters.

url = 'https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/item-search?d.status='+status+'&f.source='+urllib.parse.quote(source)

r2 = requests.get(url, headers=headers).json()
items =[]
for page in range(1, r2['pages']+1): # page numbers start from 1
    
    # The value of the 'page' parameter in the URL must be changed, otherwise
    # the same set of items is returned in every iteration
    
    dataclarin = requests.get(url+'&page='+str(page), headers=headers).json()
    
    number_items = len(dataclarin['items']) #good
    for x in range(0, number_items):
        items.append(dataclarin['items'][x])
    
df = pd.DataFrame(items)
df.head()

#### The code below seems ok! 

Just one suggestion, a faster code to iterate over the dataframe would be:

    for item in df.itertuples():
        ID = item.persistentId
        url = 'https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/datasets/'+ID
        ...
 
When possible it is better to use 'itertuples()' over 'index'

In [ ]:
bearer=mpdata.checkCredentials()
headers = {'Authorization':bearer}
for row in df.index:
    ID = df['persistentId'][row]
    url = 'https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/datasets/'+ID
    #delete_item = requests.delete(url, headers=headers)
    #print(delete_item)